In [8]:
import os
import shutil

def rename_and_copy_images(input_path, output_path, prefix='sd_114_'):
    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)
    
    # Loop through all files in the input directory
    for file in os.listdir(input_path):
        # Check if the file is a TIFF image
        if file.endswith('.tiff'):
            # Define the old and new file paths
            old_path = os.path.join(input_path, file)
            new_filename = prefix + file
            new_path = os.path.join(output_path, new_filename)
            
            # Copy and rename the file to the output directory
            shutil.copy(old_path, new_path)
            print(f"Copied and renamed: {old_path} -> {new_path}")

# Example usage
input_path = r'C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated'
output_path = r'C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated'
rename_and_copy_images(input_path, output_path)


Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B02-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B02-T01.tiff
Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B03-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B03-T01.tiff
Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B04-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B04-T01.tiff
Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B05-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B05-T01.tiff
Copied and renamed: C:\Users\k54739\Today_data\single_dose_categarized\RBT 01.14 Day10\untreated\B06-T01.tiff -> C:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_114_B06-T01.tiff


In [ ]:
import tifffile as tiff

def check_image_layers(image_dir):
    # Get all TIFF images in the directory
    image_files = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.tiff')]
    
    # Loop through each image and check its layers
    for img_path in image_files:
        image = tiff.imread(img_path)
        
        # Check if the image has exactly 3 layers (channels)
        if image.shape[0]  3:
            print(f"Warning: Image {img_path} has less than 3 layers. (Found {image.shape[0]} layers)")

# Example usage
image_dir = r"../../Day10_drugscreened&singledose_untreated"
check_image_layers(image_dir)

In [11]:
import tifffile as tiff

def check_image_layers(image_dir):
    # Get all TIFF images in the directory
    image_files = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.tiff')]
    
    # Loop through each image and check its layers
    for img_path in image_files:
        image = tiff.imread(img_path)
        
        # Check if the image has exactly 3 layers (channels)
        if image.shape[0] < 3:
            print(f"Warning: Image {img_path} has less than 3 layers. (Found {image.shape[0]} layers)")

# Example usage
image_dir = r"../../Day10_drugscreened&singledose_untreated"
check_image_layers(image_dir)

In [2]:
import os
import numpy as np
import tifffile as tiff

def replace_with_3_sharpest_layers(image_dir):
    # Get all TIFF images in the directory
    image_files = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.tiff')]
    
    # Process each image
    for img_path in image_files:
        # Read the image
        image = tiff.imread(img_path)
        
        # Check the number of layers and dimensions of the image
        if image.ndim == 2:  # Image is 2D, likely a single-channel image
            print(f"Warning: Image {img_path} is single-channel (dimensions: {image.shape}).")
            continue  # Skip processing for this image
        elif image.shape[0] < 3:  # Fewer than 3 layers
            print(f"Warning: Image {img_path} has less than 3 layers. (Found {image.shape[0]} layers)")
            continue  # Skip processing for this image

        # If the image has more than 3 layers, proceed
        if image.shape[0] > 3:
            print(f"Processing {img_path} with {image.shape[0]} layers.")
            
            # Normalize the 16-bit image to [0, 1] for consistent sharpness calculation
            image_normalized = image.astype(np.float32) / 65535.0
            
            # Calculate sharpness for each layer
            sharpness_scores = []
            for i in range(image.shape[0]):
                layer = image_normalized[i]
                gy, gx = np.gradient(layer)  # Compute gradients
                gnorm = np.sqrt(gx**2 + gy**2)  # Gradient magnitude
                sharpness = np.average(gnorm)  # Average sharpness
                sharpness_scores.append((sharpness, i))  # Store with index
            
            # Sort layers by sharpness score and select the top 3 sharpest layers
            sharpest_layers = sorted(sharpness_scores, reverse=True, key=lambda x: x[0])[:3]
            top_3_indices = [index for _, index in sharpest_layers]
            
            # Create a new image with only the 3 sharpest layers
            new_image = image[top_3_indices, :, :]
            
            # Save the new image to replace the original
            tiff.imwrite(img_path, new_image.astype(np.uint16))
            print(f"Replaced {img_path} with the 3 sharpest layers.")

# Example usage
image_dir = r"C:\Users\k54739\Bibi_new_thesis\thesis\control_day7_day10\day10"
replace_with_3_sharpest_layers(image_dir)


Processing C:\Users\k54739\Bibi_new_thesis\thesis\control_day7_day10\day10\RBTDS_4.2_D07-T01.tiff with 5 layers.
Replaced C:\Users\k54739\Bibi_new_thesis\thesis\control_day7_day10\day10\RBTDS_4.2_D07-T01.tiff with the 3 sharpest layers.
Processing C:\Users\k54739\Bibi_new_thesis\thesis\control_day7_day10\day10\RBTDS_4.2_D08-T01.tiff with 5 layers.
Replaced C:\Users\k54739\Bibi_new_thesis\thesis\control_day7_day10\day10\RBTDS_4.2_D08-T01.tiff with the 3 sharpest layers.
Processing C:\Users\k54739\Bibi_new_thesis\thesis\control_day7_day10\day10\RBTDS_4.2_D09-T01.tiff with 5 layers.
Replaced C:\Users\k54739\Bibi_new_thesis\thesis\control_day7_day10\day10\RBTDS_4.2_D09-T01.tiff with the 3 sharpest layers.
Processing C:\Users\k54739\Bibi_new_thesis\thesis\control_day7_day10\day10\RBTDS_4.2_D10-T01.tiff with 5 layers.
Replaced C:\Users\k54739\Bibi_new_thesis\thesis\control_day7_day10\day10\RBTDS_4.2_D10-T01.tiff with the 3 sharpest layers.
Processing C:\Users\k54739\Bibi_new_thesis\thesis\co

In [16]:
path = r'C:\Users\k54739\Bibi_new_thesis\thesis\Day10_drugscreened&singledose_untreated\ds_41_E11-T01.tiff'   #we removed this image because it only have one channel.
imageds = tiff.imread(path)

In [17]:
imageds.shape

(2054, 2456)

In [1]:
import os
import numpy as np
import tifffile as tiff

def single_channel_conversion(in_image_dir, out_image_dir):
    # Ensure output directory exists
    os.makedirs(out_image_dir, exist_ok=True)
    
    # Loop over all files in the input directory
    for filename in os.listdir(in_image_dir):
        # Check if the file is a TIFF image
        if filename.endswith(".tiff") or filename.endswith(".tif"):
            in_image_path = os.path.join(in_image_dir, filename)
            out_image_path = os.path.join(out_image_dir, filename)
            
            # Read the input image
            image = tiff.imread(in_image_path)

            # Ensure the image has 3 layers (channels)
            #if image.shape[0] != 3:
                #raise ValueError(f"Image {in_image_path} does not have exactly 3 layers.")
            
            # Normalize the 16-bit image to [0, 1]
            image_normalized = image.astype(np.float32) / 65535.0
            
            # Calculate sharpness for each layer
            sharpness_scores = []
            for i in range(3):
                layer = image_normalized[i]
                gy, gx = np.gradient(layer)
                gnorm = np.sqrt(gx**2 + gy**2)
                sharpness = np.average(gnorm)
                sharpness_scores.append(sharpness)
            
            # Find the index of the sharpest layer
            sharpest_layer_index = np.argmax(sharpness_scores)
            sharpest_layer_image = image[sharpest_layer_index]

            # Save the sharpest layer as a new image (scaled back to 16-bit)
            tiff.imwrite(out_image_path, (sharpest_layer_image).astype(np.uint16))

            print(f"Processed {filename}, saved sharpest layer to {out_image_path}")

# Example usage
in_image_dir = r"../../Day10_drugscreened&singledose_untreated"
out_image_dir = r"../../single_Day10_drugscreened&singledose_untreated"
single_channel_conversion(in_image_dir, out_image_dir)


Processed ds_41_D07-T01.tiff, saved sharpest layer to ../../single_Day10_drugscreened&singledose_untreated\ds_41_D07-T01.tiff
Processed ds_41_D08-T01.tiff, saved sharpest layer to ../../single_Day10_drugscreened&singledose_untreated\ds_41_D08-T01.tiff
Processed ds_41_D09-T01.tiff, saved sharpest layer to ../../single_Day10_drugscreened&singledose_untreated\ds_41_D09-T01.tiff
Processed ds_41_D10-T01.tiff, saved sharpest layer to ../../single_Day10_drugscreened&singledose_untreated\ds_41_D10-T01.tiff
Processed ds_41_D11-T01.tiff, saved sharpest layer to ../../single_Day10_drugscreened&singledose_untreated\ds_41_D11-T01.tiff
Processed ds_41_E07-T01.tiff, saved sharpest layer to ../../single_Day10_drugscreened&singledose_untreated\ds_41_E07-T01.tiff
Processed ds_41_E08-T01.tiff, saved sharpest layer to ../../single_Day10_drugscreened&singledose_untreated\ds_41_E08-T01.tiff
Processed ds_41_E09-T01.tiff, saved sharpest layer to ../../single_Day10_drugscreened&singledose_untreated\ds_41_E09-T

In [3]:
path = r'C:\Users\k54739\Bibi_new_thesis\thesis\single_Day10_drugscreened&singledose_untreated\ds_41_D07-T01.tiff'
image = tiff.imread(path)
image.shape

(2054, 2456)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader, random_split

import torch.nn.functional as F

import tifffile as tiff
import os
import time


In [6]:
import os
import tifffile as tiff

in_image_dir = r'C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10'
    
# Loop over all files in the input directory
for filename in os.listdir(in_image_dir):
    # Check if the file is a TIFF image
    if filename.endswith(".tiff") or filename.endswith(".tif"):
        in_image_path = os.path.join(in_image_dir, filename)
        out_image_path = os.path.join(out_image_dir, filename)
            
        # Read the input image
        image = tiff.imread(in_image_path)
        print(image.shape)
        continue
        # Check if the image has exactly 3 layers (channels)
        #if image.shape[0] != 3:
            #print(f"Image {filename} does not have exactly 3 layers.")
            #continue


In [7]:
import os

in_image_dir = r'C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10'

print("Files in directory:")
for filename in os.listdir(in_image_dir):
    print(filename)


Files in directory:
Group1
Group2
Group3
Group4
Group5
Group6
Group7
Group8
Group9


In [16]:

in_image_dir = r'C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10'

# Recursively traverse all subdirectories
for root, dirs, files in os.walk(in_image_dir):
    for filename in files:
        # Check if the file is a TIFF image
        if filename.lower().endswith(".tiff") or filename.lower().endswith(".tif"):
            in_image_path = os.path.join(root, filename)
            
            # Debug: Print the file path being processed
            #print(f"Processing file: {in_image_path}")
            
            
            # Read the input image
            image = tiff.imread(in_image_path)
                
                # Debug: Print the shape of the image
            #print(f"Successfully read {filename}: shape = {image.shape}")
            # Check if the image has exactly 3 layers (channels)
# Check the dimensions of the image
            if len(image.shape) == 3:
                    # Check for 3 channels (either first or last dimension)
                if image.shape[0] == 3 or image.shape[-1] == 3:
                    print(f"Image {filename} has exactly 3 layers.")
                if image.shape[0] > 3 or image.shape[-1] > 3:
                    
                    print(f"Image  has exactly {image.shape} shape.")
            elif len(image.shape) == 2:
                    # Handle 2D grayscale images
                print(f"Image {filename} is a 2D image (shape: {image.shape}).")
            else:
                print(f"Image {filename} has an unexpected shape (shape: {image.shape}).")
            continue
            

Image D10.tif is a 2D image (shape: (2054, 2456)).
Image D11.tif is a 2D image (shape: (2054, 2456)).
Image D7.tif is a 2D image (shape: (2054, 2456)).
Image D8.tif is a 2D image (shape: (2054, 2456)).
Image D9.tif is a 2D image (shape: (2054, 2456)).
Image E10.tif is a 2D image (shape: (2054, 2456)).
Image E11.tif is a 2D image (shape: (2054, 2456)).
Image E7.tif is a 2D image (shape: (2054, 2456)).
Image E8.tif is a 2D image (shape: (2054, 2456)).
Image E9.tif is a 2D image (shape: (2054, 2456)).
Image F07.tif is a 2D image (shape: (2054, 2456)).
Image F08.tif is a 2D image (shape: (2054, 2456)).
Image F09.tif is a 2D image (shape: (2054, 2456)).
Image F10.tif is a 2D image (shape: (2054, 2456)).
Image F11.tif is a 2D image (shape: (2054, 2456)).
Image G07.tif is a 2D image (shape: (2054, 2456)).
Image G08.tif is a 2D image (shape: (2054, 2456)).
Image G09.tif is a 2D image (shape: (2054, 2456)).
Image G10.tif is a 2D image (shape: (2054, 2456)).
Image G11.tif is a 2D image (shape: (

In [17]:

in_image_dir = r'C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10'

# Recursively traverse all subdirectories
for root, dirs, files in os.walk(in_image_dir):
    for filename in files:
        # Check if the file is a TIFF image
        if filename.lower().endswith(".tiff") or filename.lower().endswith(".tif"):
            in_image_path = os.path.join(root, filename)
            
            # Debug: Print the file path being processed
            #print(f"Processing file: {in_image_path}")
            
            
            # Read the input image
            image = tiff.imread(in_image_path)
                
                # Debug: Print the shape of the image
            #print(f"Successfully read {filename}: shape = {image.shape}")
            # Check if the image has exactly 3 layers (channels)
# Check the dimensions of the image
            if len(image.shape) == 3:
                    # Check for 3 channels (either first or last dimension)
                if image.shape[0] > 3 or image.shape[-1] > 3:
                    
                    print(f"Image  has exactly {image.shape} shape.")
 
            continue
            

Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456) shape.
Image  has exactly (3, 2054, 2456)

In [21]:

in_image_dir = r'C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10'
out_image_dir = r'C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10_all_3_channel'

# Recursively traverse all subdirectories
for root, dirs, files in os.walk(in_image_dir):
    for filename in files:
        # Check if the file is a TIFF image
        if filename.lower().endswith(".tiff") or filename.lower().endswith(".tif"):
            in_image_path = os.path.join(root, filename)
            
            # Debug: Print the file path being processed
            #print(f"Processing file: {in_image_path}")
            
            
            # Read the input image
            image = tiff.imread(in_image_path)
            
            
            if len(image.shape) == 2:
                    # Handle 2D grayscale images
                print(f"Image {in_image_path} is a 2D image (shape: {image.shape}).")
            
                continue
            

Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\D10.tif is a 2D image (shape: (2054, 2456)).
1
Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\D11.tif is a 2D image (shape: (2054, 2456)).
2
Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\D7.tif is a 2D image (shape: (2054, 2456)).
3
Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\D8.tif is a 2D image (shape: (2054, 2456)).
4
Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\D9.tif is a 2D image (shape: (2054, 2456)).
5
Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\E10.tif is a 2D image (shape: (2054, 2456)).
6
Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\E11.tif

In [22]:
import os
import numpy as np
import tifffile as tiff

in_image_dir = r'C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10'
out_image_dir = r'C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10_all_3_channel'

# Ensure output directory exists
os.makedirs(out_image_dir, exist_ok=True)

# Recursively traverse all subdirectories
for root, dirs, files in os.walk(in_image_dir):
    for filename in files:
        # Check if the file is a TIFF image
        if filename.lower().endswith(".tiff") or filename.lower().endswith(".tif"):
            in_image_path = os.path.join(root, filename)
            
            # Read the input image
            image = tiff.imread(in_image_path)
            
            if len(image.shape) == 2:
                # Handle 2D grayscale images
                print(f"Image {in_image_path} is a 2D image (shape: {image.shape}).")
                
                # Convert to 3-channel by duplicating the grayscale image
                three_channel_image = np.stack([image] * 3, axis=0)  # Creates (3, H, W)
                
                # Create the corresponding output path
                out_image_path = os.path.join(out_image_dir, filename)
                
                # Save the 3-channel image
                tiff.imwrite(out_image_path, three_channel_image.astype(image.dtype))
                print(f"Saved 3-channel image to {out_image_path}")
            else:
                # If already 3 channels, save as-is
                out_image_path = os.path.join(out_image_dir, filename)
                tiff.imwrite(out_image_path, image.astype(image.dtype))
                print(f"Image {in_image_path} already has 3 channels. Saved as-is.")


Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\D10.tif is a 2D image (shape: (2054, 2456)).
Saved 3-channel image to C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10_all_3_channel\D10.tif
Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\D11.tif is a 2D image (shape: (2054, 2456)).
Saved 3-channel image to C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10_all_3_channel\D11.tif
Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\D7.tif is a 2D image (shape: (2054, 2456)).
Saved 3-channel image to C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10_all_3_channel\D7.tif
Image C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10\Group1\D8.tif is a 2D image (shape: (2054, 2456)).
Saved 3

In [26]:
imag = tiff.imread(r'C:\Users\k54739\Today_data\Drug Screening\Categorized experiments\RBTDS1B Day 7,10\Day10_all_3_channel\B02-T01.tiff')
imag.shape

(3, 2054, 2456)

In [6]:
in_path = r'C:\Users\k54739\Today_data\drugscrenn_categorized\RBT DS 4.1 Day7,10\Day10\Group1\E11-T01.tiff'
out_path = r'C:\Users\k54739\Today_data\drugscrenn_categorized\RBT DS 4.1 Day7,10\Day10\threeChannel_E11-T01.tiff'
image = tiff.imread(in_path)
print(image.shape)

(2054, 2456)


In [7]:
three_channel_image = np.stack([image] * 3, axis=0)  # Creates (3, H, W)
tiff.imwrite(out_path, three_channel_image.astype(image.dtype))

In [8]:
p = r'C:\Users\k54739\Today_data\drugscrenn_categorized\RBT DS 4.1 Day7,10\Day10\threeChannel_E11-T01.tiff'
im = tiff.imread(p)
im.shape

(3, 2054, 2456)

In [22]:
in_path = r'c:\Users\k54739\Today_data\day7to_day10\drug_screen_day7to_day10\RBTDS 6.1 Day7,10\Day10\F2.tif'
out_path = r'c:\Users\k54739\Today_data\day7to_day10\drug_screen_day7to_day10\RBTDS 6.1 Day7,10\Day10\F2_3channel.tif'
image = tiff.imread(in_path)
print(image.shape)

(2054, 2456)


In [23]:
three_channel_image = np.stack([image] * 3, axis=0)  # Creates (3, H, W)
tiff.imwrite(out_path, three_channel_image.astype(image.dtype))

In [24]:
im = tiff.imread(out_path)
im.shape

(3, 2054, 2456)

In [ ]:
in_path = r'c:\Users\k54739\Today_data\day7to_day10\drug_screen_day7to_day10\RBTDS 6.1 Day7,10\Day10\Group6\F03-T01.tiff'
out_path = r'c:\Users\k54739\Today_data\day7to_day10\drug_screen_day7to_day10\RBTDS 6.1 Day7,10\Day10\Group6\F03-T01.jpeg'
image = tiff.imread(in_path)

In [31]:
import cv2

# Input and output paths
in_path = r'c:\Users\k54739\Today_data\Day10_drugscreened&singledose_untreated\sd_19_C06-T01.tiff'
out_path = r'c:\Users\k54739\Downloads\sd_19_c06.jpeg'

# Load the TIFF image
image = cv2.imread(in_path, cv2.IMREAD_UNCHANGED)

# Check if the image was successfully loaded
if image is None:
    print(f"Failed to load image from {in_path}")
else:
    print(f"Image loaded successfully with shape {image.shape} and dtype {image.dtype}")
    
    # Normalize 16-bit images for saving as JPEG if necessary
    if image.dtype == 'uint16':
        # Scale the image to 8-bit
        image = (image / 256).astype('uint8')

    # Save the image as JPEG
    cv2.imwrite(out_path, image)
    print(f"Image saved as JPEG to {out_path}")


Image loaded successfully with shape (2054, 2456) and dtype uint16
Image saved as JPEG to c:\Users\k54739\Downloads\sd_19_c06.jpeg


In [47]:
import os
import shutil

def rename_images(input_path, prefix='RBTDS1B_'):
    # Ensure the directory exists
    if not os.path.exists(input_path):
        print(f"Directory {input_path} does not exist.")
        return
    
    # Loop through all files in the input directory
    for file in os.listdir(input_path): 
        # Check if the file is a TIFF image
        if file.lower().endswith('.tiff') or file.lower().endswith(".tif"):
            # Full path of the original file
            old_path = os.path.join(input_path, file)
            
            # Create the new file name with the prefix
            new_filename = prefix + file
            
            # Full path of the renamed file
            new_path = os.path.join(input_path, new_filename)
            
            # Rename the file
            os.rename(old_path, new_path)
            print(f"Renamed: {old_path} -> {new_path}")

# Input directory
input_path = r'c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10'

# Call the function
rename_images(input_path)


Renamed: c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\D10.tif -> c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\RBTDS1B_D10.tif
Renamed: c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\D11.tif -> c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\RBTDS1B_D11.tif
Renamed: c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\D7.tif -> c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\RBTDS1B_D7.tif
Renamed: c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\D8.tif -> c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\RBTDS1B_D8.tif
Renamed: c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\D9.tif -> c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\RBTDS1B_D9.tif
Renamed: c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\E10.tif -> c:\Users\k54739\Today_data\day7to_day10_final\control\RBTD

In [48]:
im = tiff.imread(r'c:\Users\k54739\Today_data\day7to_day10_final\control\RBTDS1B\day10\RBTDS1B_D07-T01.tiff')
im.shape

(3, 2054, 2456)

In [49]:
import os

def compare_folders(day7_path, day10_path):
    # List all files in day7 and day10 directories
    day7_files = set(os.listdir(day7_path))
    day10_files = set(os.listdir(day10_path))
    
    # Find images in day7 without a match in day10
    unmatched_in_day10 = day7_files - day10_files
    
    # Check if all files in day7 have corresponding files in day10
    if not unmatched_in_day10:
        print("All images in day7 have corresponding images in day10.")
    else:
        print("The following images in day7 do not have a match in day10:")
        for filename in unmatched_in_day10:
            print(f"- {filename}")

# Paths to day7 and day10 folders
day7_path = r'c:\Users\k54739\Today_data\day7to_day10_final\control\day7'
day10_path = r'c:\Users\k54739\Today_data\day7to_day10_final\control\day10'

# Compare the folders
compare_folders(day7_path, day10_path)


All images in day7 have corresponding images in day10.


In [ ]:
image = tiff.imread(img_path)
        
        # Check the number of layers and dimensions of the image
        if image.ndim == 2:  # Image is 2D, likely a single-channel image
            print(f"Warning: Image {img_path} is single-channel (dimensions: {image.shape}).")
            continue  # Skip processing for this image
        elif image.shape[0] < 3:  # Fewer than 3 layers
            print(f"Warning: Image {img_path} has less than 3 layers. (Found {image.shape[0]} layers)")
            continue  # Skip processing for this image

        # If the image has more than 3 layers, proceed
        if image.shape[0] > 3:
            print(f"Processing {img_path} with {image.shape[0]} layers.")
            
            # Normalize the 16-bit image to [0, 1] for consistent sharpness calculation
            image_normalized = image.astype(np.float32) / 65535.0
            
            # Calculate sharpness for each layer
            sharpness_scores = []
            for i in range(image.shape[0]):
                layer = image_normalized[i]
                gy, gx = np.gradient(layer)  # Compute gradients
                gnorm = np.sqrt(gx**2 + gy**2)  # Gradient magnitude
                sharpness = np.average(gnorm)  # Average sharpness
                sharpness_scores.append((sharpness, i))  # Store with index
            
            # Sort layers by sharpness score and select the top 3 sharpest layers
            sharpest_layers = sorted(sharpness_scores, reverse=True, key=lambda x: x[0])[:3]
            top_3_indices = [index for _, index in sharpest_layers]
            
            # Create a new image with only the 3 sharpest layers
            new_image = image[top_3_indices, :, :]
            
            # Save the new image to replace the original
            tiff.imwrite(img_path, new_image.astype(np.uint16))
            print(f"Replaced {img_path} with the 3 sharpest layers.")